# A Test Case for Refactoring a Basic Portfolio Optimization Workflow

We recreated a basic portfolio optimization workflow from 
functions that will eventually be called from an automated pipeline.
Then compared to a past project using assets from the 
S&P 500 to ensure proper implimentation. 
We then replaced the current statistical estimators,
which assume a normal distribution, 
with robust estimators like those used in past projects through 202410-202411.
Finally we considered a slight variation that uses a bayesian approch 
to incorperate expectations derived from other sources.
Eventaully the functions called here will become a key part of the 
inital Condor Funds web-based minimally viable prototype.

The goal here is to identify:
* create seperate functions to support a basic portfolio optimization workflow
* call them from this project to comapre against a past project to ensure concistancy
* add alternate stratagies from past projects and compare
* add portfolio class and asset specific stat meta props
* add an alternate bayesian optimization approch and compare 

### *Notice*
*We stress that none of our Jupyter Notebook projects are to be considered final and of publication quality unless otherwise stated. We also provide no warranty or guarantee of any kind. These projects are meant for testing code and learning concepts in a transparent, and often evolving manner. If new methodologies, stratagies or fundamental understandings are applied in the future, they will most likely be explored in future projects. For transparency and documentation, old projects will not be removed but instead will be deprecated (and clearly marked as such). Look for updates on GitHub https://github.com/Rtasseff/condor_test/tree/main/project*

## Setup

In [1]:
# data set
datasetName = 'sp500_combined.csv'

# Paths

# Analytics dir path *USER SET*
analyticsDir = '/Users/rtasseff/projects/condor_test/analytics'
# Data dir path *USER SET*
dataDir = '/Users/rtasseff/projects/tmp/data_analytics_v1'
import sys
# adding analytics to the system path
sys.path.insert(0, analyticsDir)

from data_mining import load
from functions import genStats
from functions import genFin
from functions import assetPreassess as apa
from functions import utils


import numpy as np
import matplotlib.pyplot as plt


## Background 

Educational Source:
https://ocw.mit.edu/courses/15-401-finance-theory-i-fall-2008/dd628e151309a7f23962b1a31b9356e5_MIT15_401F08_lec13.pdf

*** need to review all text ***
Modern Portfolio Theory (MPT) is a concept in Finance that describes ways of diversifying and allocating assets in a financial portfolio in order to maximize the portfolio's expected return given the owner's risk tolerance. American economist Harry Markowitz first introduced MPT in a 1952 paper. The theory was intended to eliminate ideosyncratic risk, which is the risk inherent in a particular investment due to its unique characteristics.

*** At some point want to stress that mathmatically we ae using volitility as a statistical measure for 'risk', but the use of risk has some implied meaning that is not covered by this analysis. then further expand on what it might mean in a later example ***

A key component of this framework is diversification. When using MPT, an investor bundles different types of investments together so that when some of the securities fall in value, other securities rise in equal amount. Thus, the overall portfolio stays even but as markets rise overall, the portfolio rises along with the market's inside tide.

MPT argues that any given investment's risk and return characteristics should not be viewed alone but evaluated by how it affects the overall portfolio's risk and return. That is, an investor can construct a portfolio of multiple assets that will result in greater returns without a higher level of risk. As an alternative, starting with a desired level of expected return, the investor can construct a portfolio with the lowest possible risk that is capable of producing that return.

MPT uses precise financial mathematics to carefully construct the portfolio. The steps involved included:
1. portfolio selection - valuing and choosing the securities that might be included in the portfolio;
2. portfolio optimization - calculating the optimal allocation, or percentage, for each of the selected assets (i.e. choosing the right mix of assets to maximize return and/or minimize risk);
3. portfolio rebalancing - after the portfolio is implimented (i.e. assets are purchased) the value of individual assets will change over time and the holdings of various assets will need to be decreased or increased to ensure the original allocation (i.e. to maintain the correct overall percentage of each asset); 
4. portfolio reallocation - periodically it is important to check on the behaviour of individual assets as changes to the asset allocations may needed.

*** Went off on a tangent here, need to focus this ***
In the analysis here, we focused on step (2). Other steps will be convered in future projects and Condor Funds materials. It is worth pointing out that step (4) is of important practicale concern. Since the analysis here is enterly dependant on historical data, future events may change the statistical estiamtes of this data as they ocurr. In addition, there are many other market factors to consider for investment choices. The importance of these factors depends on the preferences of the individual investor. In particular, the time-frame for investment and the size (number of different assets) of the portfolio impact the liklyhood that observed returns and risks mirror historical ones. Longer investment time-frames and bigger portfolios (e.g. a 'buy-and-hold' straetegy over 10+ years on a well diversifed portfolio) allow for noise and abnormal events to smooth out, or correct themselves, over time. In these cases, reallocation could be done by simply repeating step (2). Shorter time frames (e.g. < 5 years) become increasingly reliant on other market insights not considered here. Of course there is no guarentee on any time-frame, as past performance does not always mean future results. For example, a new technology could drive a once strong company into bankruptcy over the course of a decade. In this case there would be no chance of recovery over time. However, there are stratagies accessible to even novice investors for periodically (sometimes without effort) repeating step (1) as well, but these will not be discussed here. 


Valuing the securities that might be included in the portfolio.
Calculating the desired asset allocation, that is, the mix of assets.
Performing calculations to optimize the portfolio to get the maximum amount of return for the minimum amount of risk.
Using financial analysis to monitor the portfolio to see if it meets expectations and then making changes to the individual securities or asset mix when market warrant a change.
An important consideration in MPT is that based on statistical measures such as variance and correlation, a single investment's performance is less important than how it impacts the entire portfolio.

MPT also assumes that investors are risk-averse, meaning that they prefer a less risky portfolio to a riskier one for a given level of return. As a practical matter, risk aversion implies that most people should invest in multiple asset classes (stocks, bonds, commodities, cash equivalents or cryptocurrencies for example).

## Data

In [2]:
# Read saved date base


# get all data
assets = load.multiAssetHist_CSV(dataDir+'/'+datasetName)
print('Full data set of assets')
assets
assets.info


Full data set of assets


<bound method DataFrame.info of Symbol               A        AAL        AAPL        ABBV        ABNB  \
Date                                                                    
2014-04-01   36.972244  35.767616   17.015982   34.212463         NaN   
2014-04-02   37.175602  35.550785   17.044250   35.026749         NaN   
2014-04-03   37.129684  35.201973   16.926130   35.131809         NaN   
2014-04-04   36.454021  34.466633   16.707170   34.278126         NaN   
2014-04-07   35.653683  33.731285   16.444855   33.240589         NaN   
...                ...        ...         ...         ...         ...   
2024-03-22  147.210846  14.820000  172.046646  176.798874  167.860001   
2024-03-25  145.323914  14.920000  170.618591  176.878128  167.990005   
2024-03-26  144.175781  14.920000  169.480133  177.532028  167.389999   
2024-03-27  147.130981  15.300000  173.075241  178.681305  166.410004   
2024-03-28  145.273987  15.350000  171.247726  180.415100  164.960007   

Symbol            

In [3]:
# get subset

# select stocks of interest by symbol list *USER SET*
stocklist = ["MSFT", "NEE", "CVX"]

assetsSub = assets[stocklist]

print('----------------')
print('Subset of assets')
print('----------------')

assetsSub.head()

prices, dates, syms = load.df2np(assetsSub)

print('')
print('- Prices:')
print(prices)

print('')
print('- Dates:')
print(dates)

print('')
print('- Symbols:')
print(syms)

----------------
Subset of assets
----------------

- Prices:
[[ 35.0114975   18.32213402  78.05315399]
 [ 34.95233917  18.16374969  78.28272247]
 [ 34.6649437   18.27191544  78.07941437]
 ...
 [420.89080811  61.43000031 155.27000427]
 [420.67120361  63.79000092 156.3500061 ]
 [419.9624939   63.90999985 157.74000549]]

- Dates:
['2014-04-01T00:00:00.000000000' '2014-04-02T00:00:00.000000000'
 '2014-04-03T00:00:00.000000000' ... '2024-03-26T00:00:00.000000000'
 '2024-03-27T00:00:00.000000000' '2024-03-28T00:00:00.000000000']

- Symbols:
['MSFT' 'NEE' 'CVX']


In [4]:
# optimization

# compare against old results
period=1
returns = genFin.returns(prices,period=period,metric='Log')
# use for spot check
print(np.log(34.95233917/35.0114975))
print(returns)
returnDates = dates[period:]

returnExps, returnDisps = genFin.calc_return_prop(returns,'Normal')

print(returnExps)
print(returnDisps**2)
print(returnDates)
print(len(returns))

"""RTasseff reviewed code and found differences this, and an old code example.
In old example code all rows containing nan values were removed, 
which removes data for all symbols for that date if one symbol has nan.
One example of a major impact this has is that Air BnB was added in 2023.
So there are nans for ABNB for all times before 2023.
A direct comparision to the new workflow could not be made.
We decided against rewriting anything here or there for the sole purpose of 
comparision.
The old code was only used as an example of the method,
no actianable analysis was ever done or planned, it was just a learning example.
We see no reason to spend more time on it.
We did some hand spot checking on the returned values here.
One example is shown.  
More were done in ipython, manually, but not shown here.
We determined this plus previous tests was sufficent to continue.
"""

-0.0016911120634433268
[[-0.00169111 -0.00868201  0.00293686]
 [-0.00825649  0.00593737 -0.00260048]
 [-0.02819188  0.00432457 -0.00201845]
 ...
 [-0.00286558 -0.0190267  -0.00769874]
 [-0.0005219   0.03769813  0.00693156]
 [-0.00168613  0.00187939  0.00885102]]
[0.00098787 0.00049677 0.00027974]
[0.00028903 0.00022456 0.00035058]
['2014-04-02T00:00:00.000000000' '2014-04-03T00:00:00.000000000'
 '2014-04-04T00:00:00.000000000' ... '2024-03-26T00:00:00.000000000'
 '2024-03-27T00:00:00.000000000' '2024-03-28T00:00:00.000000000']
2515


'RTasseff reviewed code and found differences this, and an old code example.\nIn old example code all rows containing nan values were removed, \nwhich removes data for all symbols for that date if one symbol has nan.\nOne example of a major impact this has is that Air BnB was added in 2023.\nSo there are nans for ABNB for all times before 2023.\nA direct comparision to the new workflow could not be made.\nWe decided against rewriting anything here or there for the sole purpose of \ncomparision.\nThe old code was only used as an example of the method,\nno actianable analysis was ever done or planned, it was just a learning example.\nWe see no reason to spend more time on it.\nWe did some hand spot checking on the returned values here.\nOne example is shown.  \nMore were done in ipython, manually, but not shown here.\nWe determined this plus previous tests was sufficent to continue.\n'

In [5]:

# calc need stat props

# set from 202411_apa_SP500
month=21
sampInt=20


returns = genFin.returns(prices,month,'Relative')
returnDates = dates[month:]

# get sampling subset
inds = np.arange(0,len(returns),sampInt)
returns_samp = returns[inds]
returnDates_samp = returnDates[inds]

returnExps, returnDisps = genFin.calc_return_prop(returns_samp,'Normal')
returnCoDispsSq = genFin.returnCoDispSq(returns_samp,'Normal')
print('Expected Returns')
print(returnExps)
print('Variation in Returns')
print(returnDisps**2) # this should be very near the diagonal of the codispssq
print('Co-Variation in Returns')
print(returnCoDispsSq)


Expected Returns
[0.02116498 0.01326033 0.00917267]
Variation in Returns
[0.00385584 0.00356294 0.00748348]
Co-Variation in Returns
[[0.00388693 0.00169858 0.00162529]
 [0.00169858 0.00359167 0.00116302]
 [0.00162529 0.00116302 0.00754383]]


In [15]:
# -- testing new functions for opt --

# get portfolio properties
m = len(returnExps)
w = np.ones(m) * (1/m)
returnExp_port, returnDisp_port = genFin.port_perform(w, returnExps, returnCoDispsSq)

# mannual spot check with above
print('-- Portfolio Props, Monthly --')
print(returnExp_port)
print(returnDisp_port)

returnExp_port, returnDisp_port = genFin.port_perform(w, returnExps, returnCoDispsSq, annualizeBy='M')

# mannual spot check with above
print('-- Portfolio Props, Annualized --')
print(returnExp_port)
print(returnDisp_port)

# test to capture errors
print('-- Portfolio Props, Errors --')

# catch weight error
w = np.array([.9,.1,.00001])
try:
    returnExp_port, returnDisp_port = genFin.port_perform(w, returnExps, returnCoDispsSq)
except Exception as exc:
    print(exc)

# size mis match
w = np.ones(m+1) * (1/(m+1))
try:
    returnExp_port, returnDisp_port = genFin.port_perform(w, returnExps, returnCoDispsSq)
except Exception as exc:
    print(exc)




# Function to calculate negative Sharpe ratio

-- Portfolio Props, Monthly --
0.014532659735113574
0.05163569059196007
-- Portfolio Props, Annualized --
0.1743919168213629
0.17887127917836224
-- Portfolio Props, Errors --
Weights in w do not add to one: 1.00001
operands could not be broadcast together with shapes (3,) (4,) 


In [ ]:





# Function to maximize Sharpe ratio
def maxSR(meanReturns, covMatrix, riskFreeRate=.03, constraintSet=(0, 1), annualize=True):
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate, annualize)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constraintSet for asset in range(numAssets))
    result = sc.minimize(negativeSR, numAssets * [1. / numAssets], args=args,
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return result

# Function to calculate portfolio variance
def portfolioVariance(weights, meanReturns, covMatrix, annualize=True):
    return portfolioPerformance(weights, meanReturns, covMatrix, annualize)[1]

# Function to minimize portfolio variance
def minimizeVariance(meanReturns, covMatrix, constraintSet=(0, 1), annualize=True):
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, annualize)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constraintSet for asset in range(numAssets))
    result = sc.minimize(portfolioVariance, numAssets * [1. / numAssets], args=args,
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return result

# Function to calculate portfolio return
def portfolioReturn(weights, meanReturns, covMatrix, annualize=True):
    return portfolioPerformance(weights, meanReturns, covMatrix, annualize)[0]

# Function to optimize the portfolio for a target return
def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0, 1), annualize=True):
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, annualize)
    constraints = ({'type': 'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix, annualize) - returnTarget},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple(constraintSet for asset in range(numAssets))
    effOpt = sc.minimize(portfolioVariance, numAssets * [1. / numAssets], args=args,
                         method='SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

# Function to calculate results for plotting
def calculatedResults(meanReturns, covMatrix, riskFreeRate=.03, constraintSet=(0, 1), annualize=True):
    maxSR_Portfolio = maxSR(meanReturns, covMatrix, riskFreeRate, constraintSet, annualize)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix, annualize)
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=["allocation"])
    maxSR_allocation.allocation = [round(i * 100, 0) for i in maxSR_allocation.allocation]

    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix, constraintSet, annualize)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix, annualize)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=["allocation"])

    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 100)
    efficientAllocations = []
    for target in targetReturns:
        opt_result = efficientOpt(meanReturns, covMatrix, target, constraintSet, annualize)
        efficientList.append(opt_result['fun'])
        efficientAllocations.append(opt_result['x'])

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns, efficientAllocations

# Function to plot the efficient frontier
def EF_graph(meanReturns, covMatrix, riskFreeRate=.03, constraintSet=(0, 1), annualize=True):
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns, efficientAllocations = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet, annualize)

    # Max Sharpe Ratio trace
    MaxSharpeRatio = go.Scatter(
        name="Maximum Sharpe Ratio",
        mode="markers",
        x=[round(maxSR_std * 100, 2)],
        y=[round(maxSR_returns * 100, 2)],
        marker=dict(
            color="red",
            size=14,
            line=dict(
                width=3,
                color="black")),
        text=[f"Return: {round(maxSR_returns * 100, 2)}%, Volatility: {round(maxSR_std * 100, 2)}%<br>{maxSR_allocation.to_string()}"],
        hoverinfo='text'
    )

    # Min Volatility trace
    MinVol = go.Scatter(
        name="Minimum Volatility",
        mode="markers",
        x=[round(minVol_std * 100, 2)],
        y=[round(minVol_returns * 100, 2)],
        marker=dict(
            color="green",
            size=14,
            line=dict(
                width=3,
                color="black")),
        text=[f"Return: {round(minVol_returns * 100, 2)}%, Volatility: {round(minVol_std * 100, 2)}%<br>{minVol_allocation.to_string()}"],
        hoverinfo='text'
    )

    # Efficient Frontier trace
    efficient_text = [
        f"Return: {round(ret * 100, 2)}%, Volatility: {round(vol * 100, 2)}%<br>" +
        "<br>".join([f"{symbol}: {round(weight * 100, 2)}%" for symbol, weight in zip(meanReturns.index, alloc)])
        for ret, vol, alloc in zip(targetReturns, efficientList, efficientAllocations)
    ]

    EF_curve = go.Scatter(
        name="Efficient Frontier",
        mode="lines",
        x=[round(vol * 100, 2) for vol in efficientList],
        y=[round(ret * 100, 2) for ret in targetReturns],
        line=dict(color="black", width=3, dash="dashdot"),
        text=efficient_text,
        hoverinfo='text'
    )
    
#      # Risk-Free Asset point
#     rf_point = go.Scatter(
#         name="Risk-Free Asset",
#         mode="markers",
#         x=[0],  # Assuming risk-free rate has zero volatility
#         y=[riskFreeRate * 100],
#         marker=dict(
#             color="blue",
#             size=14,
#             line=dict(
#                 width=3,
#                 color="black")),
#         text=f"Risk-Free Rate: {round(riskFreeRate * 100, 2)}%",
#         hoverinfo='text'
#     )
    
#     # Capital Allocation Line (CAL)
#     cal_x = [0, round(maxSR_std * 100, 2)]
#     cal_y = [riskFreeRate * 100, round(maxSR_returns * 100, 2)]
#     CAL = go.Scatter(
#         name="Capital Allocation Line",
#         mode="lines",
#         x=cal_x,
#         y=cal_y,
#         line=dict(color="orange", width=2, dash="dashdot")
#     )

    data = [MaxSharpeRatio, MinVol, EF_curve]


#     data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title='Portfolio Optimization with the Efficient Frontier',
        yaxis=dict(title='Annualized Return (%)'),
        xaxis=dict(title='Annualized Volatility (%)'),
        showlegend=True,
        legend=dict(
            x=0.75,
            y=0,
            traceorder='normal',
            bgcolor="#E2E2E2",
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()

# Example usage
# Assuming meanReturns and covMatrix are defined
# meanReturns = pd.Series([...])  # Define your mean returns
# covMatrix = pd.DataFrame([...])  # Define your covariance matrix
EF_graph(meanReturns, covMatrix)